**About :** Computes Features.

**TODO**:
- for loop to automatize

In [ ]:
cd ../src

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [ ]:
import os
import gc
import re
import cudf
import glob
import numba
import warnings
import numpy as np
import pandas as pd
import seaborn as sns

from tqdm import tqdm
from datetime import datetime
from collections import Counter
from pandarallel import pandarallel
from numerize.numerize import numerize

warnings.simplefilter(action="ignore", category=FutureWarning)
pandarallel.initialize(nb_workers=32, progress_bar=False)
pd.options.display.max_columns = 500

In [ ]:
from params import *
from data.fe import *
from utils.load import load_sessions

### Load candidates

In [ ]:
MODE = "val"  #  "val", "test"
CANDIDATES_VERSION = "c-orders-v3"
FEATURES_VERSION = "8"

SUFFIX = f"{CANDIDATES_VERSION}.{FEATURES_VERSION}"

In [ ]:
CANDIDATE_FILE = f'../output/candidates/candidates_{CANDIDATES_VERSION}_{MODE}.parquet'
PARQUET_FILES = f"../output/{MODE}_parquet/*"

if MODE == "val":
    OLD_PARQUET_FILES = "../output/full_train_parquet/*"
elif MODE == "train":
    OLD_PARQUET_FILES = "../output/other_parquet/*"
else:
    raise NotImplementedError

In [ ]:
pairs = cudf.read_parquet(CANDIDATE_FILE)
pairs = pairs.sort_values(['session', 'candidates']).reset_index(drop=True)

In [ ]:
PART = 2
CHUNK_SIZE = 1_000_000  # PER SESSION INSTEAD ??

ids = np.arange(PART * CHUNK_SIZE, min((PART + 1) * CHUNK_SIZE, len(pairs)))
pairs = pairs.iloc[ids].reset_index(drop=True)
print(pairs.shape)

### Time weighting

In [ ]:
sessions = load_sessions(PARQUET_FILES)
weights = compute_weights(sessions)

In [ ]:
pairs = pairs.merge(weights, how="left", on=["session", "candidates"])
pairs = pairs.sort_values(['session', 'candidates']).reset_index(drop=True)

for c in weights.columns[2:]:
    pairs[c] = pairs[c].fillna(pairs[c].min() / 2).astype("float32")

In [ ]:
del sessions
numba.cuda.current_context().deallocations.clear()
gc.collect()

### Popularity

In [ ]:
pairs = compute_popularity_features(pairs, [OLD_PARQUET_FILES, PARQUET_FILES], "")
pairs = compute_popularity_features(pairs, PARQUET_FILES, "_w")

In [ ]:
numba.cuda.current_context().deallocations.clear()
gc.collect()

### Covisitation features

In [ ]:
MATRIX_FOLDER = "../output/matrices/"
# MATRIX_NAMES = [f"matrix_123_temporal_20_{MODE}", f"matrix_123_type136_20_{MODE}", f"matrix_12__20_{MODE}", f"matrix_123_type0.590.5_20_{MODE}"]

In [ ]:
MATRIX_NAMES = [
    f"matrix_123_temporal_20_{MODE}",
    f"matrix_123_type136_20_{MODE}",
    f"matrix_12__20_{MODE}",
    f"matrix_123_type0.590.5_20_{MODE}",
    f"matrix_cpu-90_{MODE}",
    f"matrix_cpu-95_{MODE}",
    f"matrix_cpu-99_{MODE}",
    f"matrix_gpu-116_{MODE}",
    f"matrix_gpu-115_{MODE}",
    f"matrix_gpu-93_{MODE}",
    f"matrix_gpu-217_{MODE}",
    f"matrix_gpu-226_{MODE}",
    f"matrix_gpu-232_{MODE}",
    f"matrix_gpu-239_{MODE}",
    f"matrix_gpu-700_{MODE}",
    f"matrix_gpu-701_{MODE}",
]

In [ ]:
sessions = load_sessions(PARQUET_FILES)

sessions = sessions.sort_values(['session', "aid"]).groupby('session').agg(list).reset_index()
pairs = pairs.merge(sessions[["session", "aid"]], how="left", on="session")
pairs = pairs.sort_values(['session', 'candidates']).reset_index(drop=True)

In [ ]:
for name in MATRIX_NAMES:
    print(f' -> Features from {name}')

    fts = compute_coocurence_features(
        pairs[['session', 'candidates', 'aid']],
        os.path.join(MATRIX_FOLDER, name + ".pqt"),
        weights
    )

    for c in fts.columns[2:]:
        pairs[f"{name.rsplit('_', 1)[0]}_{re.sub('w_', '', c)}"] = fts[c].values

    del fts
    numba.cuda.current_context().deallocations.clear()
    gc.collect()
    
pairs.drop('aid', axis=1, inplace=True)

del sessions, weights
numba.cuda.current_context().deallocations.clear()
gc.collect()

### Rank ft

In [ ]:
def add_rank_feature(pairs, feature):
    df_ft = pairs[["session", "candidates", feature]]
    df_ft = df_ft.sort_values(feature, ascending=False, ignore_index=True)
    df_ft[f'{feature}_rank'] = 1
    df_ft[f'{feature}_rank'] = df_ft[f'{feature}_rank'].astype("uint8")
    df_ft[f'{feature}_rank'] = df_ft.groupby("session")[f'{feature}_rank'].cumsum()

    df_ft[f'{feature}_rank'] = df_ft.groupby(["session", feature])[f'{feature}_rank'].cummin()  # Ties

    df_ft = df_ft.drop(feature, axis=1).sort_values(["session", "candidates"], ignore_index=True)

    pairs[f'{feature}_rank'] = df_ft[f'{feature}_rank'].astype("uint8")

In [ ]:
fts_to_rank = [ft for ft in pairs.columns[5:] if not any([k in ft for k in ["_rank", "_sum", "_max"]])]

In [ ]:
for ft in tqdm(fts_to_rank):
    add_rank_feature(pairs, ft)

### Correlations

In [ ]:
# FEATURES = pairs.columns[5:]  # [ft for ft in pairs.columns[5:] if "rank" in ft]

# corr = pairs[FEATURES].corr()
# corr = corr.to_pandas()
# corr = corr.values

# mask = np.zeros_like(corr, dtype=bool)
# mask[np.triu_indices_from(mask)] = True
# corr[mask] = 0

In [ ]:
# TH = 0.99

# for i in range(len(corr)):
#     for j in range(len(corr)):
#         if corr[i, j] > TH:
#             print(FEATURES[i], FEATURES[j], f'{corr[i, j] :.3f}')

### Session features
- Count views/clicks/carts/orders of session
- Count views/clicks/carts/orders of each candidate

TODO :
- Distance to last view

In [ ]:
pairs = pairs.sort_values(['session', 'candidates']).reset_index(drop=True)

for i, c in enumerate(CLASSES + ["*"]):
    print(f'-> Candidate {c if c != "*" else "views"} in session')

    sessions = load_sessions(PARQUET_FILES)
    
    if c != "*":
        sessions.loc[sessions["type"] != i, "aid"] = -1
    sessions = sessions.groupby('session').agg(list).reset_index()

    pairs[f'candidate_{c}_before'] = count_actions(
        pairs[['session', 'candidates']],
        sessions
    )

    del sessions
    numba.cuda.current_context().deallocations.clear()
    gc.collect()
    
#     break

In [ ]:
sessions = load_sessions(PARQUET_FILES)

n_views = sessions[['session', 'ts']].groupby('session').count().reset_index().rename(columns={"ts": "n_views"})
n_clicks = sessions[sessions['type'] == 0][['session', 'ts']].groupby('session').count().reset_index().rename(columns={"ts": "n_clicks"})
n_carts = sessions[sessions['type'] == 1][['session', 'ts']].groupby('session').count().reset_index().rename(columns={"ts": "n_carts"})
n_orders = sessions[sessions['type'] == 2][['session', 'ts']].groupby('session').count().reset_index().rename(columns={"ts": "n_orders"})

sessions_fts = n_views.merge(n_clicks, how="left", on="session").fillna(0)
sessions_fts = sessions_fts.merge(n_carts, how="left", on="session").fillna(0)
sessions_fts = sessions_fts.merge(n_orders, how="left", on="session").fillna(0)

for c in sessions_fts.columns[1:]:
    sessions_fts[c] = np.clip(sessions_fts[c], 0, 255).astype(np.uint8)

In [ ]:
pairs = pairs.merge(sessions_fts, on="session", how="left")
pairs = pairs.sort_values(['session', 'candidates'])

### Save

In [ ]:
save_by_chunks(pairs, f"../output/features/fts_{MODE}_{SUFFIX}/", part=PART)

In [ ]:
pairs.head()

Done